In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

In [9]:
df = pd.read_csv("../../completed/df_stats.csv", index_col=0)

## 2.1 Analisi Statistica e Regressione (Inference)



In questa parte, l'attenzione è posta sulla significatività statistica e sull'interpretazione dei coefficienti. Non stiamo ancora cercando di fare la "miglior predizione possibile", ma di capire come le variabili si influenzano a vicenda.

**Selezione delle variabili**  
Come sappiamo il **target** è Price, e il resto sono i nostri **predittori**. Naturalmente cercheremo di investigare sulla significatività statistica di ogni coefficiente, e quindi potremmo dover cambiare alcune scelte.

Applichiamo utilizzando la libreria **statsModels** il nostro modello di **Regressione Lineare.**

In [12]:
y_inference = df['Price']

vars_da_escludere = ['Price'] 
X_inference = df.drop(columns=vars_da_escludere)

X_inference = sm.add_constant(X_inference)

modello = sm.OLS(y_inference, X_inference).fit()

print(modello.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.932
Model:                            OLS   Adj. R-squared:                  0.932
Method:                 Least Squares   F-statistic:                     2171.
Date:                Tue, 23 Dec 2025   Prob (F-statistic):               0.00
Time:                        16:39:31   Log-Likelihood:                 1362.4
No. Observations:                5710   AIC:                            -2651.
Df Residuals:                    5673   BIC:                            -2405.
Df Model:                          36                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                  -0.8504    

**1. Analisi della Bontà di Adattamento (R-squared)**  
R-squared (0.932): Il modello spiega il 93.2% della varianza totale dei prezzi. Questo indica un adattamento eccellente ai dati.  
Prob (F-statistic) (0.00): Indica che il modello nel suo complesso è statisticamente significativo (non è un risultato casuale).  
Adj. R-squared (0.932): È identico all'R-squared semplice. Significa che non abbiamo "gonfiato" il modello con variabili inutili (spazzatura). Tutte le variabili aggiunte portano informazione.

**2. Diagnostica: Il problema della Multicollinearità**  
Nelle note:  
The condition number is large, 1.64e+03. This might indicate that there are strong multicollinearity...

Avevamo ragione a temere Engine + Power. Il numero è alto, il che significa che le due variabili "litigano" un po'. Tuttavia, guardando i P-value, entrambe sono rimaste significative (0.000) e hanno coefficienti positivi.  
**Interpretazione**: Nonostante la collinearità, il modello beneficia dell'inclusione di entrambe le variabili per massimizzare la precisione (R2), sebbene l'interpretazione dei singoli coefficienti di potenza e cilindrata vada fatta con cautela.  

**3. Interpretazione dei Coefficienti**  
- A. Variabili di Usura (Age, Km)  
Age (Coef: -0.0962):  
Mantenendo costanti le altre variabili, ogni anno di invecchiamento dell'auto è associato a una riduzione del prezzo di circa il 9.6%. Questo rispecchia fedelmente il deprezzamento annuale di mercato.  
Kilometers_Driven (Coef: -0.0717):  
L'usura chilometrica ha un impatto negativo significativo: all'aumentare dei chilometri, il valore diminuisce.  

- B. Variabili di Performance (Power, Engine)  
Power (Coef: 0.6838):  
La potenza è il predittore positivo più forte. Un motore più potente è associato drasticamente a un prezzo più alto.  

- C. Variabili Categoriche (Brand e Trasmissione)
Le variabili Dummy funzionano per confronto con una base. Quale Brand manca nella lista? Audi. Quindi tutti i numeri sono rispetto ad un'Audi:  
Brand_BMW (P-value 0.201, Coef -0.02):  
Il coefficiente non è statisticamente significativo (P > 0.05). Questo significa che, a parità di età e potenza, non c'è differenza di prezzo statistica tra una BMW e un'Audi (il brand di riferimento). Il mercato le valuta uguali.  
Brand_Maruti (Coef: -0.4893):  
Il coefficiente è negativo e significativo. Rispetto al brand di riferimento (Audi), una Maruti vale circa il 49% in meno a parità di altre caratteristiche.  
Transmission (Coef: 0.0745):
Passare dal cambio Manuale (0) all'Automatico (1) comporta un incremento di prezzo del 7.5% circa, confermando il valore aggiunto della trasmissione automatica.  

**4. Analisi dei P-Value**  
Promossi (Significativi, P < 0.05): Age, Km, Fuel, Transmission, Power, Engine, Seats e la maggior parte dei Brand economici (Maruti, Hyundai, ecc.). Queste variabili devono restare nel modello.  

Bocciati o Neutri (Non Significativi, P > 0.05):  
Brand_BMW (0.201), Brand_Mercedes-Benz (0.328), Location_Jaipur, Chennai, Kochi.  
Significato: Questo non vuol dire che "non contano", ma che non sono diverse dalla media o dal riferimento (Audi). E inoltre ha senso che BMW e Mercedes siano simili ad Audi.

**5. Analisi degli Intervalli di Confidenza ([0.025 0.975])**  
Diamo una visione generale su quelli più rilevanti:  
Guardiamo la riga di Age:  
Coefficiente stimato: -0.0962. Intervallo: [-0.099, -0.094].  
Commento: L'intervallo è strettissimo (i due numeri sono vicinissimi). Questo indica una grandissima precisione della stima. Siamo estremamente certi che il deprezzamento annuo sia tra il 9.4% e il 9.9%.  
Guardiamo la riga di Brand_BMW:  
Intervallo: [-0.057, 0.012].  
Commento: L'intervallo contiene lo zero (va da negativo a positivo). Ecco perché il P-value è alto, non siamo sicuri se il brand BMW aggiunga o tolga valore rispetto alla base. Statisticamente, l'effetto è nullo.  
Power: L'intervallo è [0.645, 0.723].  
Interpretazione: Anche per la potenza la stima è solida. L'effetto positivo sul prezzo è netto e ben definito.  

La maggior parte degli intervalli di confidenza sono ristretti e non includono lo zero, il che indica che le stime dei coefficienti sono precise e affidabili. Le uniche eccezioni, riguardano brand di lusso comparabili al riferimento o location specifiche, dove l'effetto prezzo è statisticamente trascurabile.